<a href="https://colab.research.google.com/github/msakarvadia/Machine-Learning/blob/master/Naive_Bayes_Classificaiton_Document_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Probability and Statistics for AI & Machine Learning**
## Document Classification using Naive Bayes Classifier.

 NC State University


This code classifies the sentiment of IMDB movie reviews using a Naive Bayes Classifier. Because of the nature of this classifier our accuracy for classificiations will only hit around 80-85% due to our "naive" assumption that there are no dependencies between words in a moive review.

# Dataset 

In [1]:
from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=None,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step


In [2]:
"""print(x_test[10:40])"""

'print(x_test[10:40])'

In [0]:
import numpy as np
import pandas as pd
import math
import time

In [4]:
start_time = time.time()
#determine total number of words in all positive documents and total words in negative documents
oneWordCount = 0
zeroWordCount = 0
for i in range(25000):
  if y_train[i]==1:
    oneWordCount += len(x_train[i])
  else:
    zeroWordCount += len(x_train[i])
  
      
print(oneWordCount)
print(zeroWordCount)

3019537
2948304


In [0]:
#2 column array of [word, frequency] of one classified documents
oneWordFreq = {}
zeroWordFreq = {}

#determine the fequency of each word in 1 classified documents and 0 classified documents and append values to frequency array defined above
for i in range(25000):
  if y_train[i] == 1:
    for w in range(len(x_train[i])):
      oneWordFreq[x_train[i][w]] = oneWordFreq.get(x_train[i][w],0)+1
  if y_train[i] ==0:
    for w in range(len(x_train[i])):
      zeroWordFreq[x_train[i][w]] = zeroWordFreq.get(x_train[i][w],0)+1

In [6]:
#checks to make sure that the frequency counter is accuratly counting frequencies by summing all of them up and seeing if they match the total word count
total = 0
a=0
for k,v in oneWordFreq.items():
  total += v
  a+=1
print(total)

totalTwo = 0
for k,v in zeroWordFreq.items():
  totalTwo += v
print(totalTwo)

3019537
2948304


In [0]:
#In order to account for words in the test set that arnt in the training set we will use a laplace function
#add 10,000 to the total word count; this is our assumption for the amount of vocab words possible
oneWordCount += 10000
zeroWordCount += 10000

#to complete the laplace funciton, we add one to the frequency of of all words
for k,v in oneWordFreq.items():
  oneWordFreq[k]= v+1
  
for k,v in zeroWordFreq.items():
  zeroWordFreq[k]= v+1

In [0]:
#we must now assume that the probablity of words occuring are independant...this is an untrue and naive assumption but we need this to simplfy things and use conditional probability
#P(word|Classification) = Freq of word + 1 / Total # of words + 10000

for k,v in oneWordFreq.items():
  oneWordFreq[k]= v/oneWordCount
for a,b in zeroWordFreq.items():
  zeroWordFreq[a] = b/zeroWordCount

In [9]:
"""print(sorted(oneWordFreq.items(), key=lambda s: s[0]))"""

'print(sorted(oneWordFreq.items(), key=lambda s: s[0]))'

In [10]:
"""print(sorted(zeroWordFreq.items(), key=lambda s: s[0]))"""

'print(sorted(zeroWordFreq.items(), key=lambda s: s[0]))'

In [0]:
# if there is a new word in the test set, eventhough we have the laplace function, the probability will still approach zero since we are multiplying; instead we take the log of each individual probbility and add them
negChecker = 0
for k,v in oneWordFreq.items():
  oneWordFreq[k]= math.log(v)
for k,v in zeroWordFreq.items():
  zeroWordFreq[k]= math.log(v)

In [12]:
"""print(oneWordFreq.items())"""

'print(oneWordFreq.items())'

In [13]:
"""print(zeroWordFreq.items())"""

'print(zeroWordFreq.items())'

In [0]:
#for every review we calculate the total sum of all of the oneWordFreq of indepentant word probabilites and sum them up and do the same for zeroWordFreq
#compare which one is greater for each line...and voila that is your class

#we will have to still check that keys are in range and if not do 1/__WordCount

def oneTrainer(x_train, y_train):
  oneProb = []
  for i in range(25000):
      oneRowProb = 0
      zeroRowProb = 0
      for w in x_train[i]:
        word = w
        if w in oneWordFreq.keys():
          oneRowProb += oneWordFreq[word]
        else:
          oneRowProb += math.log(1/oneWordCount)
      oneProb.append(oneRowProb)
  return oneProb

In [0]:
def zeroTrainer(x_train, y_train):
  zeroProb = []
  for i in range(25000):
      oneRowProb = 0
      zeroRowProb = 0
      for w in x_train[i]:
        word = w    
        if w in zeroWordFreq.keys():
          zeroRowProb += zeroWordFreq[word]
        else:
          zeroRowProb += math.log(1/zeroWordCount)
      zeroProb.append(zeroRowProb)
  return zeroProb

In [0]:
def errorCheck(oneProb, zeroProb, y_train):
  checkerArr = []
  error = 0
  for i in range(25000):
    if oneProb[i]>zeroProb[i]:
      checkerArr.append(1)
    else:
      checkerArr.append(0)
    if y_train[i] != checkerArr[i]:
      error +=1
  return (error/25000)*100

In [17]:
#this is where we check our naive bayes classifier with test set data
one = oneTrainer(x_test, y_test)
zero = zeroTrainer(x_test, y_test)
errorCheck(one, zero, y_test)
accuracy = 0
accuracy = 100 - errorCheck(one, zero, y_test)
print("Accuracy is", accuracy, "%")
end_time = time.time()
print("Total execution time: {}".format(end_time - start_time))

Accuracy is 81.27199999999999 %
Total execution time: 16.617316246032715
